In [2]:
import tensorflow as tf
from tensorflow import Graph, Session

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers import MaxPooling3D, MaxPooling2D
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add
from scipy.io import loadmat
graph = tf.Graph()


In [3]:
# Function to create to a series of CONV layers followed by Max pooling layer
def Convblock(channel_dimension, block_no, no_of_convs) :
    Layers = []
    for i in range(no_of_convs) :
        
        Conv_name = "conv"+str(block_no)+"_"+str(i+1)
        
        # A constant kernel size of 3*3 is used for all convolutions
        Layers.append(Convolution2D(channel_dimension,kernel_size = (3,3),padding = "same",activation = "relu",name = Conv_name))
    
    Max_pooling_name = "pool"+str(block_no)
    
    #Addding max pooling layer
    Layers.append(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),name = Max_pooling_name))
    
    return Layers

In [4]:
def FCN_8_helper(image_size):
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = (image_size,image_size,3)))
    
    for l in Convblock(64,1,2) :
        model.add(l)
    
    for l in Convblock(128,2,2):
        model.add(l)
    
    for l in Convblock(256,3,3):
        model.add(l)
    
    for l in Convblock(512,4,3):
        model.add(l)
    
    for l in Convblock(512,5,3):
        model.add(l)
        
    model.add(Convolution2D(4096,kernel_size=(7,7),padding = "same",activation = "relu",name = "fc6"))
      
    #Replacing fully connnected layers of VGG Net using convolutions
    model.add(Convolution2D(4096,kernel_size=(1,1),padding = "same",activation = "relu",name = "fc7"))
    
    # Gives the classifications scores for each of the 21 classes including background
    model.add(Convolution2D(21,kernel_size=(1,1),padding="same",activation="relu",name = "score_fr"))
    
    Conv_size = model.layers[-1].output_shape[2] #16 if image size if 512
    #print(Conv_size)
    
    model.add(Deconvolution2D(21,kernel_size=(4,4),strides = (2,2),padding = "valid",activation=None,name = "score2"))
    
    # O = ((I-K+2*P)/Stride)+1 
    # O = Output dimesnion after convolution
    # I = Input dimnesion
    # K = kernel Size
    # P = Padding
    
    # I = (O-1)*Stride + K 
    Deconv_size = model.layers[-1].output_shape[2] #34 if image size is 512*512
    
    #print(Deconv_size)
    # 2 if image size is 512*512
    Extra = (Deconv_size - 2*Conv_size)
    
    #print(Extra)
    
    #Cropping to get correct size
    model.add(Cropping2D(cropping=((0,Extra),(0,Extra))))
    
    return model
    


In [5]:
output = FCN_8_helper(512)
print(len(output.layers))

24


In [12]:
def FCN_8(image_size):
    with graph.as_default():
        session1 = Session()
        with session1.as_default():
            model = Sequential()
            fcn_8 = FCN_8_helper(image_size)
            #Calculating conv size after the sequential block
            #32 if image size is 512*512
            Conv_size = fcn_8.layers[-1].output_shape[2] 

            #Conv to be applied on Pool4
            skip_con1 = Convolution2D(21,kernel_size=(1,1),padding = "same",activation=None, name = "score_pool4")

            #Addig skip connection which takes adds the output of Max pooling layer 4 to current layer
            Summed = add(inputs = [skip_con1(fcn_8.layers[14].output),fcn_8.layers[-1].output])

            #Upsampling output of first skip connection
            x = Deconvolution2D(21,kernel_size=(4,4),strides = (2,2),padding = "valid",activation=None,name = "score4")(Summed)
            x = Cropping2D(cropping=((0,2),(0,2)))(x)


            #Conv to be applied to pool3
            skip_con2 = Convolution2D(21,kernel_size=(1,1),padding = "same",activation=None, name = "score_pool3")

            #Adding skip connection which takes output og Max pooling layer 3 to current layer
            Summed = add(inputs = [skip_con2(fcn_8.layers[10].output),x])

            #Final Up convolution which restores the original image size
            Up = Deconvolution2D(21,kernel_size=(16,16),strides = (8,8),
                                 padding = "valid",activation = None,name = "upsample")(Summed)

            #Cropping the extra part obtained due to transpose convolution
            final = Cropping2D(cropping = ((0,8),(0,8)))(Up)
    
    
    return Model(fcn_8.input, final)

In [13]:
def build_model():
    with graph.as_default():
        session1 = Session()
        with session1.as_default():
            model = FCN_8(512)
    
    return model

m = build_model()  # Your model implementation

with graph.as_default():
  # compile method actually creates the model in the graph.
  m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
writer = tf.summary.FileWriter(logdir='FCN8', graph=graph)
writer.flush()

In [14]:
!tensorboard --logdir logdir

W0402 02:53:25.166716 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0402 02:53:25.166716 123145444306944 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0402 02:53:25.167817 Reloader tf_logging.py:120] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0402 02:53:25.167816 123145444306944 tf_logging.py:120] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0402 02:53:25.181649 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0402 02:53:25.181648 1231454